# IK Method with Robot Arm Control 

In [1]:
import numpy as np
import time

# Get a robotic arm objectb
Arm = Arm_Device()
time.sleep(.1)


def Skew(w):
    skew_mat = np.array([[0, -w[2], w[1]],
                                [w[2], 0, -w[0]],
                                [-w[1], w[0], 0]], dtype = object)
    return skew_mat

def matrix_exp(w_hat, theta):
    skew_w = Skew(w_hat)
    R = np.eye(3) + np.sin(theta) * skew_w + (1 - np.cos(theta)) * np.matmul(skew_w, skew_w)
    return R

def screw_exp(screw, theta):
    w_hat = screw[:3]
    v = screw[3:]

    w_hat_skew = Skew(w_hat)

    R = matrix_exp(w_hat, theta)
    G_theta = np.eye(3) * theta + (1 - np.cos(theta)) * w_hat_skew + (theta - np.sin(theta)) * np.matmul(w_hat_skew, w_hat_skew)

    T = np.vstack((np.hstack((R, np.matmul(G_theta, v.reshape((-1, 1))))),
                       [0, 0, 0, 1]))
    return T
    
def Ad(T):
    p = T[:3, 3]
    R = T[:3, :3]
    p_mat = Skew(p)
    Adjoint = np.vstack((np.hstack((R, np.zeros((3, 3)))),
                             np.hstack((p_mat @ R, R))))
    return Adjoint

def BodyTwist(V_bmat):
    w_b = np.array([V_bmat[2, 1], V_bmat[0, 2], V_bmat[1, 0]])
    w_b = w_b.reshape(-1,1)
    v_b = V_bmat[:3, 3]
    v_b = v_b.reshape(-1,1)
    output = np.vstack((w_b, v_b))
    return output

In [2]:
import numpy as np
import time

# Get a robotic arm objectb
Arm = Arm_Device()
time.sleep(.1)

# Control six servos at the same time, gradually changing the angle.
def ctrl_all_servo(angle, s_time = 500):
    Arm.Arm_serial_servo_write6(thd[0], thd[1], thd[2], thd[3], thd[4], 90, s_time)
    time.sleep(s_time/1000)

from scipy.linalg import logm

w = np.array([[0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1]])
q = np.array([[0, 0, 0], [0, 0, 0.1253], [0, 0.0834, 0.1253], [0, 0.1668, 0.1253], [0, 0.1668, 0.0753]])
v = -np.cross(w,q)

q_end = np.array([0, 0.1668, 0.0479])
M = np.vstack([np.hstack([np.eye(3), q_end.reshape(-1, 1)]),
                  np.array([0, 0, 0, 1])])

s1 = np.hstack([w[0], v[0]])
s2 = np.hstack([w[1], v[1]])
s3 = np.hstack([w[2], v[2]])
s4 = np.hstack([w[3], v[3]])
s5 = np.hstack([w[4], v[4]])

# Desired Position # Adjust these things
x_d = np.array([0, 0.12, 0.0899]) 
x_d = x_d.reshape(-1,1)
T_sd = np.vstack([np.hstack([np.eye(3), x_d]),
                  np.array([[0, 0, 0, 1]])])
th = np.deg2rad([31.9677, 44.3471, -55.6161, 11.2690, -31.9616])
th = th.reshape(-1,1)

k = 1
dt = 0.5

w_b = np.array([1, 1, 1])
w_b = w_b.reshape(-1,1)
v_b = np.array([1, 1, 1])
v_b = v_b.reshape(-1,1)

while (np.linalg.norm(w_b) > 0.001 or np.linalg.norm(v_b) > 0.0001):

    # Forward Kinematic
    exp1 = screw_exp(s1.reshape(-1,1), th[0])
    exp2 = screw_exp(s2.reshape(-1,1), th[1])
    exp3 = screw_exp(s3.reshape(-1,1), th[2])
    exp4 = screw_exp(s4.reshape(-1,1), th[3])
    exp5 = screw_exp(s5.reshape(-1,1), th[4])
    T_sb = exp1 @ exp2 @ exp3 @ exp4 @ exp5 @ M 
    T_sb = T_sb.astype(dtype = 'float64')
    
    # Inverse Kinematic
    T_bs = np.linalg.inv(T_sb)
    AdT_bs = Ad(T_bs)

    Js1 = s1.reshape(-1,1)
    Js2 = Ad(exp1) @ s2.reshape(-1,1)
    Js3 = Ad(exp1 @ exp2) @ s3.reshape(-1,1)
    Js4 = Ad(exp1 @ exp2 @ exp3) @ s4.reshape(-1,1)
    Js5 = Ad(exp1 @ exp2 @ exp3 @ exp4) @ s5.reshape(-1,1)
    Js = np.hstack([Js1, Js2, Js3, Js4, Js5])
    Js = Js.astype(dtype = 'float64')
    Jb = AdT_bs @ Js
    Jb = Jb.astype(dtype = 'float64')

    T_bd = T_bs @ T_sd
    V_bmat = logm(T_bd) * 0.07
    w_b = np.array([V_bmat[2, 1], V_bmat[0, 2], V_bmat[1, 0]])
    v_b = V_bmat[:3, 3]
    V_b = np.hstack([w_b, v_b])

    th_center = np.zeros((1, 5))
    th_center[0,2] = -np.pi / 3
    th_center[0,3] = np.pi / 6
    grad_H = np.zeros((1, 5))
    grad_H[0,2] = th[2] - th_center[0,2]
    grad_H[0,3] = th[3] - th_center[0,3]
    grad_H = grad_H.reshape(-1,1)
    gamma = -0.2

    Jvb = np.linalg.pinv(Jb) @ V_b
    Jvb = Jvb.reshape(-1,1)
    dth = Jvb + gamma * (np.eye(5) - np.linalg.pinv(Jb) @ Jb) @ grad_H

    th = th + dth*dt
    th_deg = np.rad2deg(th)

    thd = np.array([th_deg[0]+90, th_deg[1], th_deg[2]+90, th_deg[3], -th_deg[4]+90])
    
    ctrl_all_servo(thd, 500)
    
    k = k + 1

    if k == 250:
        break

NameError: name 'Arm_Device' is not defined

In [3]:
import numpy as np
from Arm_Lib import Arm_Device
# Get a robotic arm object
Arm = Arm_Device()
time.sleep(.1)

# Control six servos at the same time, gradually changing the angle.
def ctrl_all_servo(angle, s_time = 500):
    Arm.Arm_serial_servo_write6(thd[0], thd[1], thd[2], thd[3], thd[4], 90, s_time)
    time.sleep(s_time/1000)

def Skew(w):
    skew_mat = np.array([[0, -w[2], w[1]],
                                [w[2], 0, -w[0]],
                                [-w[1], w[0], 0]], dtype = object)
    return skew_mat

def matrix_exp(w_hat, theta):
    skew_w = Skew(w_hat)
    R = np.eye(3) + np.sin(theta) * skew_w + (1 - np.cos(theta)) * np.matmul(skew_w, skew_w)
    return R

def screw_exp(screw, theta):
    w_hat = screw[:3]
    v = screw[3:]

    w_hat_skew = Skew(w_hat)

    R = matrix_exp(w_hat, theta)
    G_theta = np.eye(3) * theta + (1 - np.cos(theta)) * w_hat_skew + (theta - np.sin(theta)) * np.matmul(w_hat_skew, w_hat_skew)

    T = np.vstack((np.hstack((R, np.matmul(G_theta, v.reshape((-1, 1))))),
                       [0, 0, 0, 1]))
    return T
    
def Ad(T):
    p = T[:3, 3]
    R = T[:3, :3]
    p_mat = Skew(p)
    Adjoint = np.vstack((np.hstack((R, np.zeros((3, 3)))),
                             np.hstack((p_mat @ R, R))))
    return Adjoint

def BodyTwist(V_bmat):
    w_b = np.array([V_bmat[2, 1], V_bmat[0, 2], V_bmat[1, 0]])
    w_b = w_b.reshape(-1,1)
    v_b = V_bmat[:3, 3]
    v_b = v_b.reshape(-1,1)
    output = np.vstack((w_b, v_b))
    return output

ModuleNotFoundError: No module named 'Arm_Lib'

In [4]:
import numpy as np
import time

from scipy.linalg import logm

w = np.array([[0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1]])
q = np.array([[0, 0, 0], [0, 0, 0.1253], [0, 0.0834, 0.1253], [0, 0.1668, 0.1253], [0, 0.1668, 0.0753]])
v = -np.cross(w,q)

q_end = np.array([0, 0.1668, 0.0479])
M = np.vstack([np.hstack([np.eye(3), q_end.reshape(-1, 1)]),
                  np.array([0, 0, 0, 1])])

s1 = np.hstack([w[0], v[0]])
s2 = np.hstack([w[1], v[1]])
s3 = np.hstack([w[2], v[2]])
s4 = np.hstack([w[3], v[3]])
s5 = np.hstack([w[4], v[4]])

# Desired Position # Adjust these things
x_d = np.array([-0.08, 0.12, 0.0899]) 
x_d = x_d.reshape(-1,1)
T_sd = np.vstack([np.hstack([np.eye(3), x_d]),
                  np.array([[0, 0, 0, 1]])])
th_deg = np.array([-34.9935, 40.1605, -48.3002, 8.1397, 34.9414])
th = np.deg2rad([-34.9935, 40.1605, -48.3002, 8.1397, 34.9414])
th = th.reshape(-1,1)
    
thd = [th_deg[0]+90, th_deg[1], th_deg[2]+90, th_deg[3], -th_deg[4]+90]
    
ctrl_all_servo(thd, 500)


k = 1
dt = 2

w_b = np.array([1, 1, 1])
w_b = w_b.reshape(-1,1)
v_b = np.array([1, 1, 1])
v_b = v_b.reshape(-1,1)

while (np.linalg.norm(w_b) > 0.001 or np.linalg.norm(v_b) > 0.0001):

    # Forward Kinematic
    exp1 = screw_exp(s1.reshape(-1,1), th[0])
    exp2 = screw_exp(s2.reshape(-1,1), th[1])
    exp3 = screw_exp(s3.reshape(-1,1), th[2])
    exp4 = screw_exp(s4.reshape(-1,1), th[3])
    exp5 = screw_exp(s5.reshape(-1,1), th[4])
    T_sb = exp1 @ exp2 @ exp3 @ exp4 @ exp5 @ M 
    T_sb = T_sb.astype(dtype = 'float64')
    
    # Inverse Kinematic
    T_bs = np.linalg.inv(T_sb)
    AdT_bs = Ad(T_bs)

    Js1 = s1.reshape(-1,1)
    Js2 = Ad(exp1) @ s2.reshape(-1,1)
    Js3 = Ad(exp1 @ exp2) @ s3.reshape(-1,1)
    Js4 = Ad(exp1 @ exp2 @ exp3) @ s4.reshape(-1,1)
    Js5 = Ad(exp1 @ exp2 @ exp3 @ exp4) @ s5.reshape(-1,1)
    Js = np.hstack([Js1, Js2, Js3, Js4, Js5])
    Js = Js.astype(dtype = 'float64')
    Jb = AdT_bs @ Js
    Jb = Jb.astype(dtype = 'float64')

    T_bd = T_bs @ T_sd
    V_bmat = logm(T_bd) * 0.08
    w_b = np.array([V_bmat[2, 1], V_bmat[0, 2], V_bmat[1, 0]])
    v_b = V_bmat[:3, 3]
    V_b = np.hstack([w_b, v_b])

    th_center = np.zeros((1, 5))
    th_center[0,2] = -np.pi / 3
    th_center[0,3] = np.pi / 6
    grad_H = np.zeros((1, 5))
    grad_H[0,2] = th[2] - th_center[0,2]
    grad_H[0,3] = th[3] - th_center[0,3]
    grad_H = grad_H.reshape(-1,1)
    gamma = -0.2

    Jvb = np.linalg.pinv(Jb) @ V_b
    Jvb = Jvb.reshape(-1,1)
    dth = Jvb + gamma * (np.eye(5) - np.linalg.pinv(Jb) @ Jb) @ grad_H

    th = th + dth*dt
    th_deg = np.rad2deg(th)

    thd = np.array([th_deg[0]+90, th_deg[1], th_deg[2]+90, th_deg[3], -th_deg[4]+90])
    
    ctrl_all_servo(thd, 500)
    
    k = k + 1

    if k == 400:
        break

NameError: name 'ctrl_all_servo' is not defined